In [1]:
from solinv.environment import InvariantEnvironment
from solinv.tyrell.interpreter import InvariantInterpreter
from solinv.tyrell import spec as S
from solinv.tyrell import dsl as D

from solinv.tyrell.dsl.utils import derive_dfs, get_hole_dfs

In [2]:
spec = S.parse_file("./dsls/abstract0.tyrell")
start_type = spec.get_type("Expr")
builder = D.Builder( spec )
interpreter = InvariantInterpreter()

In [3]:
config = {
    "spec": spec,
    "start_type": start_type,
    "max_step": 10,
    "contract_path": "../SolidTypes/test/regression/good/mint_MI.sol",
    "max_contract_length": 2500,
    "token_list_path": "./token_list0.pkl",
    "solc_version": "0.5.17",
    "interpreter": interpreter
}
env = InvariantEnvironment(config=config)

In [5]:
env.spinv_to_stoinv("enum2expr(<VAR2>)")

'enum2expr(<VAR2>)'

In [4]:
env.stovar_list

['_balances', '_totalSupply']

In [5]:
env.spvar_to_prod

{'<VAR0>': EnumProduction(id=7, lhs=EnumType(EnumExpr, domain=['<VAR0>', '<VAR1>', '<VAR2>', '0']), choice=0),
 '<VAR1>': EnumProduction(id=8, lhs=EnumType(EnumExpr, domain=['<VAR0>', '<VAR1>', '<VAR2>', '0']), choice=1)}

In [6]:
env.stovar_to_spvar

{'_balances': '<VAR0>', '_totalSupply': '<VAR1>'}

In [7]:
len(env.contract_tokens)

2500

In [8]:
str(env.contract_ids)

'[3, 119, 7, 2, 8, 120, 7, 3, 2, 7, 5, 2, 8, 6, 8, 4, 8, 23, 7, 2, 8, 22, 7, 121, 8, 79, 7, 5, 3, 104, 7, 5, 6, 8, 105, 7, 5, 6, 8, 106, 7, 110, 8, 68, 7, 26, 8, 107, 7, 37, 8, 23, 7, 2, 8, 108, 7, 5, 2, 8, 6, 8, 29, 7, 2, 8, 22, 7, 109, 8, 79, 7, 5, 3, 50, 7, 20, 8, 23, 7, 2, 8, 29, 7, 16, 8, 22, 7, 51, 8, 41, 7, 2, 8, 21, 7, 2, 8, 52, 7, 37, 8, 53, 7, 55, 8, 27, 7, 3, 24, 7, 164, 8, 25, 7, 165, 8, 4, 8, 46, 7, 3, 23, 7, 2, 8, 155, 7, 3, 23, 7, 2, 8, 29, 7, 30, 8, 22, 7, 48, 8, 21, 7, 2, 8, 27, 7, 3, 24, 7, 31, 8, 25, 7, 30, 8, 4, 8, 4, 8, 22, 7, 156, 8, 21, 7, 2, 8, 27, 7, 3, 24, 7, 164, 8, 25, 7, 165, 8, 4, 8, 157, 7, 3, 23, 7, 2, 8, 29, 7, 30, 8, 22, 7, 48, 8, 21, 7, 2, 8, 27, 7, 3, 24, 7, 31, 8, 25, 7, 30, 8, 4, 8, 4, 8, 4, 8, 38, 7, 26, 8, 44, 7, 47, 8, 4, 8, 3, 50, 7, 20, 8, 23, 7, 2, 8, 29, 7, 17, 8, 22, 7, 51, 8, 41, 7, 2, 8, 21, 7, 2, 8, 52, 7, 37, 8, 53, 7, 55, 8, 27, 7, 3, 24, 7, 31, 8, 25, 7, 30, 8, 4, 8, 46, 7, 3, 23, 7, 2, 8, 29, 7, 30, 8, 22, 7, 48, 8, 21, 7, 2, 8, 27, 

### vocabulary generation
iterating through a lot of contracts to determine a vocabulary; note that ideally this should come from the solc manual, but I'm in a rush

In [7]:
import os

In [9]:
solc_versions = ["0.6.12", "0.5.17", "0.4.26"]
file_list = os.listdir("/Users/joseph/Desktop/UCSB/21fall/SolidTypes/test/regression/good/")
json_list = []
for dfile in file_list:
    print("# processing {}".format(dfile))
    tmp_contract_path = "/Users/joseph/Desktop/UCSB/21fall/SolidTypes/test/regression/good/{}".format(dfile)
    for p in solc_versions:
        try:
            tmp_json = env.get_contract_ast(tmp_contract_path, p)
            json_list.append(tmp_json)
            break
        except:
            print("    solc {} failed, try another version".format(p))
            continue

# processing conversions3.sol
    solc 0.6.12 failed, try another version
    solc 0.5.17 failed, try another version
# processing mint_MMI.sol
# processing conversions2.sol
    solc 0.6.12 failed, try another version
    solc 0.5.17 failed, try another version
# processing sum_loop.sol
    solc 0.6.12 failed, try another version
    solc 0.5.17 failed, try another version
# processing mod1.sol
    solc 0.6.12 failed, try another version
    solc 0.5.17 failed, try another version
# processing conversions1.sol
    solc 0.6.12 failed, try another version
    solc 0.5.17 failed, try another version
# processing structs.sol
# processing asm.sol
# processing conversions5.sol
    solc 0.6.12 failed, try another version
    solc 0.5.17 failed, try another version
# processing if3.sol
    solc 0.6.12 failed, try another version
    solc 0.5.17 failed, try another version
# processing if2.sol
    solc 0.6.12 failed, try another version
    solc 0.5.17 failed, try another version
# processing c

In [10]:
# extract all tokens
def rec_extract_tokens(arg_json):
    if isinstance(arg_json, dict):
        ret_list = []
        for p in arg_json.keys():
            ret_list += [str(p)]
            ret_list += rec_extract_tokens(arg_json[p])
        return ret_list
    elif isinstance(arg_json, list):
        ret_list = []
        for p in arg_json:
            ret_list += rec_extract_tokens(p)
        return ret_list
    else:
        return [str(arg_json)]

from collections import Counter
token_counter = Counter({})
for tmp_json in json_list:
    token_counter.update(rec_extract_tokens(tmp_json))

In [20]:
# filter vocabulary
tmp0_tokens = []
for p in token_counter.most_common():
    if p[1] <= 10:
        # not so frequent, skip
        continue
    
    try:
        # skip all numbers
        float(p[0])
        continue
    except:
        pass
    
    # variable name too short, skip, e.g., x, y, z, a
    if len(p[0]) <= 1 and p[0].lower() in "abcdefghijklmnopqrstuvwxyz":
        continue
        
    # if you reach here, add it to the vocabulary
    tmp0_tokens.append(p[0])

In [23]:
import pickle
with open("./token_list0.pkl", "wb") as f:
    pickle.dump(tmp0_tokens, f)

In [6]:
import torch

In [8]:
a = torch.rand(5,4,3)

In [10]:
a.permute(1,0,2).size()

torch.Size([4, 5, 3])

In [13]:
a[:,-1:,:].size()

torch.Size([5, 1, 3])

In [16]:
torch.rand(1, 256).squeeze(0).size()

torch.Size([256])